In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install bnlp-toolkit

In [3]:
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier

from bnlp import NLTKTokenizer
import math
import random

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
rand_seed = 1601
np.random.seed(rand_seed)
random.seed(rand_seed)

In [5]:
df_train = pd.read_excel('/content/drive/Shareddrives/TFQ/SentNoB_After_Def/noise_train.xlsx')
df_test = pd.read_excel('/content/drive/Shareddrives/TFQ/SentNoB_After_Def/noise_test.xlsx')

In [6]:
col_name = ['Slang', 'Local', 'Word Misuse', 'Context/Word Missing', 'Wrong Serial',
            'Mixed Language', 'Punctuation', 'Space Error', 'Spelling', 'Internet Garbage']

for col in col_name:
    df_train[col] = pd.to_numeric(df_train[col], errors='coerce')
    df_test[col] = pd.to_numeric(df_test[col], errors='coerce')

df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [7]:
X_train = df_train['Data'].values
X_train = [str(text) for text in X_train]

y_train = df_train[col_name].values

X_test = df_test['Data'].values
X_test = [str(text) for text in X_test]

y_test = df_test[col_name].values

In [8]:
tfidf_vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 4))
# tfidf_vectorizer_word = TfidfVectorizer(analyzer='word', ngram_range=(1, 4))

# combined_features = FeatureUnion([
#     ('char_tfidf', tfidf_vectorizer),
#     ('word_tfidf', tfidf_vectorizer_word)
# ])

svm_model = LinearSVC(random_state=rand_seed)
multilabel_classifier = MultiOutputClassifier(svm_model, n_jobs=-1)

pipeline = Pipeline([
    ('tfidf', tfidf_vectorizer),
    # ('tfidf', combined_features),
    ('clf', multilabel_classifier),
])

In [9]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(analyzer='char', ngram_range=(1, 4))),
                ('clf',
                 MultiOutputClassifier(estimator=LinearSVC(random_state=1601),
                                       n_jobs=-1))])

In [10]:
predictions = pipeline.predict(X_test)

In [11]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.91      0.61      0.73       127
           1       0.66      0.42      0.51       152
           2       0.67      0.05      0.09        82
           3       0.33      0.02      0.04        50
           4       0.00      0.00      0.00         4
           5       0.79      0.81      0.80       538
           6       0.84      0.39      0.53       881
           7       0.85      0.13      0.23       429
           8       0.63      0.51      0.57       609
           9       1.00      0.64      0.78        47

   micro avg       0.76      0.45      0.57      2919
   macro avg       0.67      0.36      0.43      2919
weighted avg       0.77      0.45      0.53      2919
 samples avg       0.59      0.44      0.48      2919

